In [80]:
#nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/patsnap/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [64]:
from __future__ import absolute_import, division, print_function

In [62]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import keras
import nltk
import spacy

Using TensorFlow backend.


In [63]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from gensim import logging
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as  pd
import seaborn as sns
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import MWETokenizer

In [11]:
import json
import logging
import sys
import pandas as pd
#convert trial data to spacy format
df = pd.read_fwf('/Users/patsnap/Desktop/Neo4J_and_other_codes/SPacy_trial_data/8184428/train.txt',header =None)
df.columns = ['Words']
df['Words'] = df['Words'].str.split(" ")
df = pd.DataFrame(df.Words.values.tolist()).add_prefix('code_')
df = df.drop(['code_2','code_3'], axis=1)

In [12]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [100]:
df.columns = ['words','labels']
punctuations = [x for x in string.punctuation]
stop_words = set(stopwords.words('english'))

for index, row in df.iterrows():
    if row['words'] in punctuations:
        row['labels'] == 'O'

df.head()

,words,labels
0,variable,O
1,temperature,O
2,electron,B-CMT
3,paramagnetic,I-CMT
4,resonance,I-CMT


In [14]:
df_check_O = df[df.labels == 'O']
df_check_O.head(2)

,words,labels
0,variable,O
1,temperature,O


In [39]:
label_list = df.labels.tolist()
label_list =list(set(label_list))
label_list

['I-DSC',
 'O',
 'I-PRO',
 'B-SMT',
 'B-MAT',
 'I-SPL',
 'I-CMT',
 'I-SMT',
 'B-APL',
 'B-DSC',
 'B-SPL',
 'B-CMT',
 'B-PRO',
 'I-APL',
 'I-MAT']

In [38]:
df.labels = df.labels.replace("B-MA","B-MAT")
df.labels = df.labels.replace("B-M","B-MAT")
df.labels = df.labels.replace("I-MA","I-MAT")
df.labels = df.labels.replace("I-M","I-MAT")
df.labels = df.labels.replace("I-DS","I-DSC")
df.labels = df.labels.replace("B-DS","B-DSC")
df.labels = df.labels.replace("I-D","I-DSC")
df.labels = df.labels.replace("B-D","B-DSC")
df.labels = df.labels.replace("I-PR","I-PRO")
df.labels = df.labels.replace("B-PR","B-PRO")
df.labels = df.labels.replace("I-P","I-PRO")
df.labels = df.labels.replace("B-P","B-PRO")
df.labels = df.labels.replace("I-AP","I-APL")
df.labels = df.labels.replace("B-AP","B-APL")
df.labels = df.labels.replace("I-A","I-APL")
df.labels = df.labels.replace("B-A","B-APL")
df.labels = df.labels.replace("I-SM","I-SMT")
df.labels = df.labels.replace("B-SM","B-SMT")
df.labels = df.labels.replace("I-S","I-SMT")
df.labels = df.labels.replace("B-S","B-SMT")
df.labels = df.labels.replace("I-CM","I-CMT")
df.labels = df.labels.replace("B-CM","B-CMT")
df.labels = df.labels.replace("I-C","I-CMT")
df.labels = df.labels.replace("B-C","B-CMT")
df.labels = df.labels.replace("B-C","B-CMT")
df.labels = df.labels.replace("B-","O")
df.labels = df.labels.replace("I-","O")
df.labels = df.labels.replace("B","O")
df.labels = df.labels.replace("I","O")
df.labels = df.labels.replace("1/2","O")
df.labels = df.labels.replace("","O")
df.labels = df.labels.fillna("O")

In [101]:
inorganic_material = df[(df.labels == 'B-MAT') | (df.labels == 'I-MAT')]
inorganic_material = inorganic_material.words.tolist()
inorganic_material = [x for x in inorganic_material if x not in punctuations]
inorganic_material = [x for x in inorganic_material if x not in stop_words]
sample_descripter = df[(df.labels == 'B-DSC') | (df.labels == 'I-DSC')]
sample_descripter = sample_descripter.words.tolist()
sample_descripter = [x for x in sample_descripter if x not in punctuations]
sample_descripter = [x for x in sample_descripter if x not in stop_words]
material_property = df[(df.labels == 'B-PRO') | (df.labels == 'I-PRO')]
material_property = material_property.words.tolist()
material_property = [x for x in material_property if x not in punctuations]
material_property = [x for x in material_property if x not in stop_words]
material_application = df[(df.labels == 'B-APL') | (df.labels == 'I-APL')]
material_application = material_application.words.tolist()
material_application = [x for x in material_application if x not in punctuations]
material_application = [x for x in material_application if x not in stop_words]
synthesis_method = df[(df.labels == 'B-SMT') | (df.labels == 'I-SMT')]
synthesis_method = synthesis_method.words.tolist()
synthesis_method = [x for x in synthesis_method if x not in punctuations]
synthesis_method = [x for x in synthesis_method if x not in stop_words]
characterization_method = df[(df.labels == 'B-CMT') | (df.labels == 'I-CMT')]
characterization_method = characterization_method.words.tolist()
characterization_method = [x for x in characterization_method if x not in punctuations]
characterization_method = [x for x in characterization_method if x not in stop_words]

In [65]:
#remove duplicates
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [102]:
inorganic_material = ordered_set(inorganic_material)
sample_descripter = ordered_set(sample_descripter)
material_property = ordered_set(material_property)
material_application = ordered_set(material_application)
synthesis_method = ordered_set(synthesis_method)
characterization_method = ordered_set(characterization_method)

In [54]:
df.to_csv('/Users/patsnap/Desktop/Neo4J_and_other_codes/SPacy_trial_data/8184428/train_bert.tsv', sep = '\t',header=False, index =False)
unknown_label_list = ["None"]
def tsv_to_json_format(input_path,output_path,unknown_label_list):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity not in unknown_label_list:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None
tsv_to_json_format("/Users/patsnap/Desktop/Neo4J_and_other_codes/SPacy_trial_data/8184428/train_bert.tsv",'/Users/patsnap/Desktop/Neo4J_and_other_codes/SPacy_trial_data/8184428/train_bert.json',unknown_label_list)

In [55]:
training_data = []
lines=[]
with open("/Users/patsnap/Desktop/Neo4J_and_other_codes/SPacy_trial_data/8184428/train_bert.json", 'r') as f:
    lines = f.readlines()
for line in lines:
    data = json.loads(line)
    text = data['content']
    training_data.append(text)

In [76]:
scientific_data = pd.DataFrame(training_data)
scientific_data = scientific_data.dropna()
scientific_data = scientific_data.reset_index(drop=True)
scientific_data.columns = ['text']
scientific_data.head()

,text
0,variable temperature electron paramagnetic res...
1,"the peak - to - peak linewidth ( DHPP ) , g fa..."
2,"furthermore , these parameters ( DHPP , g fact..."
3,"the DHPP , g factor and NS decreased with incr..."
4,characterization of CuInSe2 thin films produce...


In [103]:
sd_list = []
for x in inorganic_material:
    sd_list.append(x)
for x in sample_descripter:
    sd_list.append(x)
for x in material_property:
    sd_list.append(x)
for x in material_application:
    sd_list.append(x)
for x in synthesis_method:
    sd_list.append(x)
for x in characterization_method:
    sd_list.append(x)

sd_list = ordered_set(sd_list)

my_idx = {}
for i,w in enumerate(sd_list):
    my_idx[" ".join(w)] = i
print(len(my_idx))

4522


In [104]:
mwe = MWETokenizer(sd_list,separator=' ')
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt))
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [200]:
from tqdm import tqdm_notebook as tqdm
all_item = []

for i in tqdm(range(len(scientific_data))):
    word_ls = []
    
    for token in spans(scientific_data.text[i]):
 
        my_tuple = token[0]
        #print("###",token)
       
        #my_tuples = ' , '.join(map(str, my_tuple))
        if token[0] in inorganic_material:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]         
            tag_list = ['I-MAT']*len(pos_list)
            tag_list[0] = 'B-MAT' 
                       
            for s,p,t in zip(subwords,pos_list,tag_list):           
                if type(p) == list:
                    p = p[0][1]
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)                    
             
        elif token[0] in material_property:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-PRO']*len(pos_list)
            tag_list[0] = 'B-PRO' 
            
             
            for s,p,t in zip(subwords,pos_list,tag_list):   
                if type(p) == list:
                    p = p[0][1]
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)
        
        elif token[0] in material_application:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-APL']*len(pos_list)
            tag_list[0] = 'B-APL' 
            
             
            for s,p,t in zip(subwords,pos_list,tag_list):   
                if type(p) == list:
                    p = p[0][1]
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)
                
        elif token[0] in synthesis_method:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-SMT']*len(pos_list)
            tag_list[0] = 'B-SMT' 
            
             
            for s,p,t in zip(subwords,pos_list,tag_list):   
                if type(p) == list:
                    p = p[0][1]
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)
        
        elif token[0] in characterization_method:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-CMT']*len(pos_list)
            tag_list[0] = 'B-CMT' 
            
             
            for s,p,t in zip(subwords,pos_list,tag_list):   
                if type(p) == list:
                    p = p[0][1]
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)
                
        elif token[0] in sample_descripter:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-DSC']*len(pos_list)
            tag_list[0] = 'B-DSC' 
            
             
            for s,p,t in zip(subwords,pos_list,tag_list):   
                if type(p) == list:
                    p = p[0][1]
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)               
             
        else:
            #print(type(my_tuple))
            my_pos = nltk.pos_tag([my_tuple])[0][1]
            new_item = dict({'Sentence #': i+1, 'Tag' : 'O', 'Word': my_tuple.lower(),'POS': my_pos})
            all_item.append(new_item)
            
        

In [123]:
data = pd.DataFrame(all_item)
data.columns
data.head(5)

,Sentence #,Tag,Word,POS
0,1,B-PRO,variable,JJ
1,1,B-PRO,temperature,NN
2,1,B-PRO,electron,NN
3,1,B-PRO,paramagnetic,JJ
4,1,B-PRO,resonance,NN


In [124]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [125]:
#concat sentence
getter = SentenceGetter(data)
word_list = [ [s[0] for s in sent] for sent in getter.sentences]
sentences = word_list
sentences[0]
print(len(sentences))

3755


In [126]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(len(labels[0]))

40


In [127]:
print(len(word_list[0]))

40


In [128]:
print(len(labels))

3755


In [129]:
tags_vals = list(set(data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals)}

In [130]:
words = list(set(data["Word"].values))
n_words = len(words); 
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
n_words

8226

In [131]:
idx2tag[2]

'B-MAT'

Training with bert

In [134]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [158]:
MAX_LEN = 143
bs = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [159]:
tokenized_texts = word_list
print(tokenized_texts[0])

['variable', 'temperature', 'electron', 'paramagnetic', 'resonance', 'studies', 'of', 'the', 'NiZn', 'ferrite', '/', 'O2Si', 'nanocomposite', 'effects', 'of', 'the', 'silica', 'content', 'and', 'temperature', 'on', 'the', 'magnetic', 'properties', 'of', 'Fe4NiO8Zn', '/', 'O2Si', 'nanocomposites', 'have', 'been', 'studied', 'by', 'electron', 'paramagnetic', 'resonance', '(', 'EPR', ')', 'technique']


In [ ]:
#sentences[0]

Load embedding

In [161]:
tokens_ids = [[word2idx[w] for w in s] for s in tokenized_texts]
print(len(tokens_ids[0]),len(labels[0]))

40 40


In [162]:
print(len(tokens_ids[0]),len(labels[0]))

40 40


In [163]:
#input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
input_ids = pad_sequences(tokens_ids,
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")
#input_ids = pad_sequences([convert_word2idx(txt) for txt in tokenized_texts],
                          #maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")

In [164]:
print(MAX_LEN)
for i in tokens_ids:
    if len(i) > MAX_LEN:
        #print(tokens_ids)
        print("need more")
        MAX_LEN = len(i)
print(MAX_LEN)

143
143


In [165]:
t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
len(t_list[0])

40

In [166]:
print(input_ids[0])

[2106 6124  945 7578 7117 5695 1729 6058 7144 1316 4453 1294  231 7619
 1729 6058 5007 3040 6040 6124 5507 6058 3468 3952 1729 7690 4453 1294
 1241 4153 7791 4824 4909  945 7578 7117 2986 3825 2720 7975    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]


In [167]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")

In [168]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [169]:
#split train test
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [170]:
#change to torhc tensor
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [171]:
print(tr_inputs.shape)
print(tr_masks.shape)
print(tr_tags.shape)

torch.Size([3379, 143])
torch.Size([3379, 143])
torch.Size([3379, 143])


In [172]:
print(val_inputs.shape)
print(val_masks.shape)
print(val_tags.shape)

torch.Size([376, 143])
torch.Size([376, 143])
torch.Size([376, 143])


In [173]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [174]:
print(len(tag2idx))

7


In [175]:
model = BertForTokenClassification.from_pretrained(u"bert-base-uncased", num_labels=len(tag2idx))

In [176]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [177]:
#Fine tune BERT
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [192]:
epochs = 10
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels, true_inputs = [], [],[]
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        inputs = b_input_ids.to('cpu').numpy()
         
        true_inputs.append(inputs)
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]


    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))



Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.005642738886617392
Validation loss: 0.19988851714879274
Validation Accuracy: 0.3927192599067599




Epoch:  10%|█         | 1/10 [1:11:05<10:39:45, 4265.09s/it]

F1-Score: 0.2101881817092173
Train loss: 0.004327756324511149
Validation loss: 0.19262664268414179
Validation Accuracy: 0.4443108974358975




Epoch:  20%|██        | 2/10 [2:22:43<9:30:00, 4275.04s/it] 

...

Validation loss: 0.20123906154185534
Validation Accuracy: 0.4164481351981351
F1-Score: 0.21673586397785685
Train loss: 0.003637606643507134




Epoch:  90%|█████████ | 9/10 [10:08:11<1:02:23, 3743.10s/it]

Validation loss: 0.18374982243403792
Validation Accuracy: 0.43817380536130535
F1-Score: 0.22301038503738102
Train loss: 0.0026998859462970736
Validation loss: 0.2106392588466406
Validation Accuracy: 0.5433238636363636




Epoch: 100%|██████████| 10/10 [11:08:26<00:00, 4010.61s/it] 

F1-Score: 0.26049039811247493


In [180]:
count = 0 
all_data = 0
for i,j,ll in zip(pred_tags,valid_tags,val_inputs):
    for k,l,kk in zip(i,j,ll):
        if k==l   : 
            count += 1
            print(k,l,idx2word[kk.item()])
        all_data += 1
print(count)
print(all_data)

In [193]:
#evaluate model
model.eval()
predictions = []
true_labels = []
true_inputs = []

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
print(len(valid_dataloader))
for batch in tqdm(valid_dataloader):
    #print(len(batch))
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    inputs = b_input_ids.to('cpu').numpy()
    true_inputs.append(inputs)
    
    
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]

print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))



  0%|          | 0/24 [00:00<?, ?it/s]

24




  4%|▍         | 1/24 [00:11<04:17, 11.18s/it]

  8%|▊         | 2/24 [00:24<04:20, 11.84s/it]

 12%|█▎        | 3/24 [00:35<04:05, 11.69s/it]

 17%|█▋        | 4/24 [00:46<03:48, 11.42s/it]

 21%|██        | 5/24 [00:57<03:33, 11.23s/it]

 25%|██▌       | 6/24 [01:08<03:22, 11.23s/it]

 29%|██▉       | 7/24 [01:20<03:15, 11.49s/it]

 33%|███▎      | 8/24 [01:32<03:04, 11.52s/it]

 38%|███▊      | 9/24 [01:44<02:54, 11.60s/it]

 42%|████▏     | 10/24 [01:55<02:40, 11.46s/it]

 46%|████▌     | 11/24 [02:06<02:29, 11.49s/it]

 50%|█████     | 12/24 [02:18<02:17, 11.46s/it]

 54%|█████▍    | 13/24 [02:29<02:04, 11.33s/it]

 58%|█████▊    | 14/24 [02:40<01:52, 11.27s/it]

 62%|██████▎   | 15/24 [02:51<01:40, 11.18s/it]

 67%|██████▋   | 16/24 [03:02<01:29, 11.13s/it]

 71%|███████   | 17/24 [03:13<01:17, 11.11s/it]

 75%|███████▌  | 18/24 [03:25<01:08, 11.44s/it]

 79%|███████▉  | 19/24 [03:38<00:59, 11.97s/it]

 83%|████████▎ | 20/24 [03:52<00:49, 12.38s/it]

 88%|████████▊ | 21/24 [04:

Validation loss: 0.2106392588466406
Validation Accuracy: 0.5433238636363636
Validation F1-Score: 0.26049039811247493


In [188]:
count = 0 
all_data = 0
for i,j,ll in zip(pred_tags,valid_tags,val_inputs):
    for k,l,kk in zip(i,j,ll):
        if k==l   : 
            count += 1
            #print(k,l,idx2word[kk.item()])  #get this and add this as list of tuples to later convert it to dataframe
        all_data += 1
#print(count)
#print(all_data)